# Note:
The entire code took around one hour forty five minutes to run.

In [1]:
# import all libs here
from pymongo import MongoClient
import json
from bson import Code
import pymongo
import pprint

# Import all relevant libraries for pre-processing
# Import tokenize for splitting sentences and words
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import string

import math
import numpy as np
import pandas as pd
from time import time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\srijitac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# make connection
st = time()
client = pymongo.MongoClient("mongodb://localhost:27017/")
print("Time taken for connection : ", time() - st, "s.")

Time taken for connection :  0.010938882827758789 s.


In [3]:
# create collection litcovidSample and load data
st = time()

db = client['covidData2']
covidItems = db['test']
db.list_collection_names()
if 'test' in db.list_collection_names():
    db['test'].drop()
litcovid = db["test"]

with open('litcovid2BioCJSON_small.json') as f:
    file_data = json.load(f)
    
covidItems.insert_many(file_data)

print("Time taken to load data : ", time() - st, "s.")

Time taken to load data :  0.6961395740509033 s.


In [4]:
# Preprocessing of data

st = time()

# List of punctuations
punct = list(string.punctuation)

# Extract all the texts from the passages
texts_array = []
for y in covidItems.find():
    for x in y['passages']:
        texts_array.append(x['text'])

# Split texts_array into individual sentences
sen_array = []
for a in texts_array:
    b = sent_tokenize(a)
    sen_array.extend(b)

# Split sen_array to individual words
# Remove stop words and punctuations and digits
words_array = []
stop_words = set(stopwords.words('english'))
for a in sen_array:
    b = word_tokenize(a.lower())
    temp_array = []
    
    for x in b:        
        if (x not in stop_words) and not x.isdigit() and not x.replace('.', '', 1).isdigit() \
        and (x not in punct) and (len(x)>2) and (x not in temp_array):
            temp_array.append(x)
    if(len(temp_array) > 0 ):
        words_array.append(temp_array)

print(len(words_array))

print("Time taken to preprocess the data : ", time() - st, "s.")

78952
Time taken to preprocess the data :  14.378509283065796 s.


# Frequent Itemset Calculation

In [5]:
min_support = 200/len(words_array)
min_support

0.00253318471982977

In [6]:
unique_list = list(map(set, words_array))

In [12]:
## Creation of list of unique words from the entire words_array
def list_1_creation(words_array):
    
    list1 = []
    for word in words_array:
        for item in word:
            if not [item] in list1:
                list1.append([item])
    list1.sort()
    
    # create a set for each item in list1
    return [set(x) for x in list1]

C1 = list_1_creation(words_array)

##Creates patterns based on pattern number(1,2,3)
##Example of pattern number : if pattern_nbr = 2 then (a,b)(a,c)(b,c)
def list_n_creation(list_n, pattern_nbr):
    
    final_list = []
    len_list = len(list_n)
    
    # join sets if first pattern_nbr-2 items are equal
    for i in range(len_list):
        for j in range(i+1, len_list):
            L1 = list(list_n[i])[:pattern_nbr-2]
            L2 = list(list_n[j])[:pattern_nbr-2]
            L1.sort()
            L2.sort()
            if L1==L2:
                final_list.append(list_n[i] | list_n[j])
    
    return final_list

##scanD = freq_itemset_calculater
## data = set_data
## Ck = freq_itemset
def freq_itemset_calculater(set_data, freq_itemset, min_support):
    """
    Scan through transaction data and return a list of candidates that meet
    the support threshold, and support data about the current candidates.
    
    Arguments:
       set_data: data set,
       freq_itemset: a list of candidate sets
       min_support: the minimum support
    """
    count = {}
    len_d = float(len(unique_list))
    ##Create dictionry with word and its frequency
    for item in set_data:
        for freq_word in freq_itemset:
            if freq_word.issubset(item):
                fw = frozenset(freq_word)
                if fw not in count:
                    count[fw] = 1                
                else:
                    count[fw] += 1
                    num_items = len_d

    num_items = len_d
    final_list = []
    support_data = {}
    support_data_transposed = {}
    
    # calculate support for every itemset in count dictionary
    for key in count:
        support_value = count[key]/num_items      
        
        # If the support meets the minimum support requirements, add it to the lists.
        if support_value >= min_support:
            final_list.insert(0, key)
            support_data[key] = support_value
            support_data_transposed[support_value] = key
    
    return final_list, support_data, support_data_transposed

In [13]:
st = time()

list1 = list_1_creation(words_array)

print("Time taken to create first frequent itemset : ", time() - st, "s.")

Time taken to create first frequent itemset :  253.4085648059845 s.


In [14]:
st = time()

L1, support_data1, support_data1_tansposed = freq_itemset_calculater(unique_list, list1, min_support)

print("Time taken to find first frequent itemset : ", time() - st, "s.")

Time taken to find first frequent itemset :  285.6554272174835 s.


In [15]:
L = L1
C = C1
itr = 2

#list of frequent items
freq_itemset_lists = []
#list of patterns
pattern_lists = []
#list of frequent items with their support values
support_value_lists = []
#list of support values with their frequent items
support_value_transposed_lists = []
#list of unique support values
unique_support_lists = []

freq_itemset_lists.append(L)
pattern_lists.append(C)
support_value_lists.append(support_data1)
support_value_transposed_lists.append(support_data1_tansposed)

for key, item in support_data1.items():
    unique_support_lists.append(item)
            
while len(L) != 0:
    
    st = time()
    
    C = list_n_creation(L, itr)
    L, support_data, support_data_tansposed = freq_itemset_calculater(unique_list, C, min_support)
    
    if len(L) != 0:
        freq_itemset_lists.append(L)
        pattern_lists.append(C)
        support_value_lists.append(support_data)
        support_value_transposed_lists.append(support_data_tansposed)
        for key, item in support_data.items():
            unique_support_lists.append(item)
            
    itr = itr + 1
    
    print("Time taken to find frequent itemset ", itr-1, " : ", time() - st, "s.")

Time taken to find frequent itemset  2  :  1670.1855034828186 s.
Time taken to find frequent itemset  3  :  12.286511182785034 s.
Time taken to find frequent itemset  4  :  0.10970878601074219 s.
Time taken to find frequent itemset  5  :  0.03989291191101074 s.
Time taken to find frequent itemset  6  :  0.01495051383972168 s.
Time taken to find frequent itemset  7  :  0.007018566131591797 s.


In [16]:
unique_support_set = set(unique_support_lists)

In [17]:
dictionary = {}
for i in support_value_transposed_lists:
    
    for key, value in i.items():
        if key in dictionary:
        
            if not isinstance(dictionary[key], list):                
                dictionary[key] = [dictionary[key]]
            
            dictionary[key].append(value)
        else:
            dictionary[key] = value

# Finding Closed Pattern 

In [18]:
# Close patterns and top 5 close patterns 

close_patterns = []
one_close_pattern_sup = []
one_close_pattern = []
two_close_pattern_sup = []
two_close_pattern = []

for dicts in support_value_lists:
    for itemset, support in dicts.items():
        is_close_pattern = True        
        f_itemsets = itemset
        f_support = support
        f_length = len(itemset)
        item_sets_to_check = dictionary[f_support]
        for it in item_sets_to_check:
            if (f_itemsets != it):
                if(frozenset.issubset(f_itemsets, it)):
                    is_close_pattern = False
                    break
        if(is_close_pattern):
            close_patterns.append(itemset)
            if f_length == 1:
                one_close_pattern.append(itemset)
                one_close_pattern_sup.append(support)
            if f_length == 2:
                two_close_pattern.append(itemset)
                two_close_pattern_sup.append(support)
            
print("TOTAL CLOSED PATTERNS : " + str(len(close_patterns)))
print()

print("TOP 5 2-ITEMSET CLOSED PATTERNS :")
indexes = sorted(range(len(two_close_pattern_sup)), key=lambda i: two_close_pattern_sup[i])[-5:]
indexes.reverse()
print()
for i in range(0, len(indexes)):
    print("Itemset: ", str(list(two_close_pattern[indexes[i]])))
    print("Support: ", str(math.ceil(two_close_pattern_sup[indexes[i]] * len(words_array))))
    print()

TOTAL CLOSED PATTERNS : 1144

TOP 5 2-ITEMSET CLOSED PATTERNS :

Itemset:  ['patients', 'covid-19']
Support:  3240

Itemset:  ['pandemic', 'covid-19']
Support:  1905

Itemset:  ['covid-19', 'disease']
Support:  1367

Itemset:  ['study', 'covid-19']
Support:  1087

Itemset:  ['coronavirus', 'disease']
Support:  1045

